In [1]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 5.2 MB/s eta 0:00:00


In [2]:
!pip install -U transformers trl accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requir

In [3]:
!pip install -U huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [4]:
import os
import torch
from trl import SFTTrainer
from datasets import load_dataset
from peft import LoraConfig, PeftConfig
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging)


In [5]:
model_identifier="microsoft/Phi-3-mini-4k-instruct"
source_dataset="gamino/wiki_medical_terms"

In [7]:
lora_hyper_r=64
lora_hyper_alpha=16
lora_hyper_dropout=0.1

In [8]:
enable_4bit=True
compute_dtype_bnb="float16"
quant_type_bnb="nf4"
double_quant_flag=False

In [9]:
training_arguments=TrainingArguments(output_dir="./results", per_device_train_batch_size=1,gradient_accumulation_steps=4,warmup_steps=10,max_steps=100,learning_rate=5e-5,fp16=True,logging_steps=1,optim="paged_adamw_8bit")


In [10]:
enable_packing = False
sequence_length_max = None
device_assignment = {"": 0}

In [11]:
dtype_computation = getattr(torch, compute_dtype_bnb)
bnb_setup = BitsAndBytesConfig(load_in_4bit = enable_4bit,
                               bnb_4bit_quant_type = quant_type_bnb,
                               bnb_4bit_use_double_quant = double_quant_flag,
                               bnb_4bit_compute_dtype = dtype_computation)

In [12]:
llama_model = AutoModelForCausalLM.from_pretrained(model_identifier, quantization_config = bnb_setup, device_map = device_assignment)
llama_model.config.use_case = False
llama_model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [13]:
print(llama_model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): Phi3RMSNorm((3072,), 

In [14]:
llama_tokenizer = AutoTokenizer.from_pretrained(model_identifier, trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"
llama_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

1

In [15]:
data = load_dataset(source_dataset)
data = data.map(lambda samples: llama_tokenizer(samples["page_title"]), batched=True)

README.md:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

wiki_medical_terms.parquet:   0%|          | 0.00/33.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6861 [00:00<?, ? examples/s]

Map:   0%|          | 0/6861 [00:00<?, ? examples/s]

In [16]:
def formatting_func(example):
    text = f"{example['page_title'][0]} {example['page_text'][0]}"
    return [text]
formatting_func(data["train"])

['Paracetamol poisoning Paracetamol poisoning, also known as acetaminophen poisoning, is caused by excessive use of the medication paracetamol (acetaminophen). Most people have few or non-specific symptoms in the first 24 hours following overdose. These include feeling tired, abdominal pain, or nausea. This is typically followed by a couple of days without any symptoms, after which yellowish skin, blood clotting problems, and confusion occurs as a result of liver failure. Additional complications may include kidney failure, pancreatitis, low blood sugar, and lactic acidosis. If death does not occur, people tend to recover fully over a couple of weeks. Without treatment, death from toxicity occurs 4 to 18 days later.Paracetamol poisoning can occur accidentally or as an attempt to die by suicide. Risk factors for toxicity include alcoholism, malnutrition, and the taking of certain other hepatotoxic medications. Liver damage results not from paracetamol itself, but from one of its metabol

In [17]:
peft_setup = LoraConfig(lora_alpha = lora_hyper_alpha,
                        lora_dropout = lora_hyper_dropout,
                        r = lora_hyper_r,
                        bias = "none",
                        task_type = "CAUSAL_LM",
                        target_modules=['o_proj','qkv_proj','gate_up_proj','down_proj'])

In [18]:
llama_sftt_trainer = SFTTrainer(model = llama_model,
                                args = training_arguments,
                                train_dataset = data["train"],
                                tokenizer = llama_tokenizer,
                                peft_config = peft_setup,
                                formatting_func=formatting_func,
                                packing = enable_packing)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:493: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a valid formatting function. Therefore `formatting_func` will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [19]:
llama_sftt_trainer.train()

You are not running the flash-attention implementation, expect numerical differences.


Step,Training Loss
1,4.248600
2,4.514700
3,4.045300
4,2.676500
5,3.940700
6,4.424200
7,4.220100
8,3.409200
9,5.801700
10,5.367800


TrainOutput(global_step=100, training_loss=3.6938879466056824, metrics={'train_runtime': 198.9155, 'train_samples_per_second': 2.011, 'train_steps_per_second': 0.503, 'total_flos': 56935723438080.0, 'train_loss': 3.6938879466056824, 'epoch': 0.058300539279988337})

In [20]:
text = "please tell about ascariases "
device = "cuda:0"
inputs = llama_tokenizer(text, return_tensors="pt").to(device)
outputs = llama_model.generate(**inputs, max_new_tokens=200)
print(llama_tokenizer.decode(outputs[0], skip_special_tokens=True))

please tell about ascariases 

Ascariasis is an infection caused by the parasitic roundworm Ascaris lumbricoides. It is one of the most common human intestinal worm infections worldwide. The infection occurs when a person ingests microscopic Ascaris eggs, which are typically found in soil contaminated with human feces or in food that has been fertilized with human manure.

**Life Cycle of Ascaris lumbricoides:**

1. **Ingestion of Eggs**: The life cycle begins when a person ingests Ascaris eggs from contaminated soil, food, or water.

2. **Hatching**: Once inside the intestine, the eggs hatch into larvae.

3. **Migration**: The larvae then penetrate the intestinal wall and enter the bloodstream.


In [21]:
from huggingface_hub import notebook_login

notebook_login()

In [22]:
llama_model.push_to_hub("my-fined-tuned-phi-3b")

model.safetensors:   0%|          | 0.00/3.23G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Saurabh2411/my-fined-tuned-phi-3b/commit/dbebf51bbb8ef6a9ca7bc23c5e9097ea7f9ba0ba', commit_message='Upload Phi3ForCausalLM', commit_description='', oid='dbebf51bbb8ef6a9ca7bc23c5e9097ea7f9ba0ba', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Saurabh2411/my-fined-tuned-phi-3b', endpoint='https://huggingface.co', repo_type='model', repo_id='Saurabh2411/my-fined-tuned-phi-3b'), pr_revision=None, pr_num=None)

In [23]:
llama_tokenizer.push_to_hub("my-fined-tuned-phi-3b")

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Saurabh2411/my-fined-tuned-phi-3b/commit/90f20ab7b2370ead88ae720335d9a37038cef7a3', commit_message='Upload tokenizer', commit_description='', oid='90f20ab7b2370ead88ae720335d9a37038cef7a3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Saurabh2411/my-fined-tuned-phi-3b', endpoint='https://huggingface.co', repo_type='model', repo_id='Saurabh2411/my-fined-tuned-phi-3b'), pr_revision=None, pr_num=None)

In [24]:
llama_sftt_trainer.push_to_hub("Saurabh2411/my-fined-tuned-adapter-phi-3b")


events.out.tfevents.1727010515.1435a8005994.751.0:   0%|          | 0.00/27.1k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/403M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Saurabh2411/results/commit/cee8ca7981e59badab4fc042c6d7c58e169c879e', commit_message='Saurabh2411/my-fined-tuned-adapter-phi-3b', commit_description='', oid='cee8ca7981e59badab4fc042c6d7c58e169c879e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Saurabh2411/results', endpoint='https://huggingface.co', repo_type='model', repo_id='Saurabh2411/results'), pr_revision=None, pr_num=None)